# 1. pad_sequence
- max seq_len에 맞춰서 padding
- link: https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch

### param QA
- batch_fifst: https://discuss.pytorch.org/t/could-someone-explain-batch-first-true-in-lstm/15402
    - `False`인 경우
        - input: seq_len, batch_size, features
        - output: seq_len, batch_size, features
    - `True`인 경우
        - input: batch_size, seq_len, features
        - output: batch_size, seq_len, features

# 2. pack_padded_sequence
- pad=0을 계산하지 않기 위한 작업
- return:
    - elems: 데이터를 flatten한 값
    - 각 time step에서 훈련할 데이터 갯수(pad 데이터 제외) 
- 참고:https://github.com/HarshTrivedi/packing-unpacking-pytorch-minimal-tutorial

In [35]:
import torch
# pad_sequence
a = [torch.tensor([1,2,3]), torch.tensor([3, 4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
b

tensor([[1, 2, 3],
        [3, 4, 0]])

In [42]:
# pack_padded_sequence
seq_len = [len(t) for t in a]
packed_input = torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True,\
                                        lengths=seq_len)

In [44]:
packed_input.data

tensor([1, 3, 2, 4, 3])

In [47]:
# 마지막에 train set batch size 1인 이유는, 
# b에서 볼 수 있듯, 데이터가 3, pad=0 이므로 
# 0 제외한 갯수 
packed_input.batch_sizes

tensor([2, 2, 1])

# 3. contiguous, repea, copy_
- link: https://discuss.pytorch.org/t/contigious-vs-non-contigious-tensor/30107/2
- link2: https://stackoverflow.com/questions/48915810/pytorch-contiguous
- 관련 에러: view operation runtime error
    - view는 discontiguous에서는 작동 안됨
    - ` invalid argument 2: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Call .contiguous() before .view().`
- stride: `Stride is the jump necessary to go from one element to the next one in the specified dimension`
    - 다음 row로 가려면; 다음 column으로 가려면 메모리 몇개를 skip해야 하는가

In [60]:
# row: 0 -> 3가려면 3개 메모리 skip
# col: 0 -> 1가려면 1개 메모리 skip
# 즉 메모리가 0,1,2,3,4,5,..이렇게 저장중
x = torch.arange(12).view(4,3)
x, x.stride()

(tensor([[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]]), (3, 1))

- view 에러 케이스
    - transpose 후 메모리는 contiguous하지 않음
    - 그상태로 flatten 시도하면 에러

In [83]:
y = x.t()

print(y.is_contiguous()) # not contiguous
print(y.stride()) # stride 바뀜!!

# error
try:
    y.view(-1)
except RuntimeError as e:
    print(e)

# solution
y = y.contiguous()
print(y)
print(y.stride())
print(y.view(-1))

False
(1, 3)
view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.
tensor([[ 0,  3,  6,  9],
        [ 1,  4,  7, 10],
        [ 2,  5,  8, 11]])
(4, 1)
tensor([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11])
